In [ ]:
import os
import subprocess
import SimpleITK as sitk
import numpy as np
import torch
from monai.transforms import RandRotate, RandGaussianNoise, RandShiftIntensity, Compose

def resample_nrrd(input_folder, output_folder, spacing=(1.0, 1.0, 1.0)):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".nrrd"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            img = sitk.ReadImage(input_path)
            resample = sitk.ResampleImageFilter()
            resample.SetOutputSpacing(spacing)
            resample.SetInterpolator(sitk.sitkLinear)
            resampled_img = resample.Execute(img)
            sitk.WriteImage(resampled_img, output_path)
    print("Resampling completed!")

def crop_image(input_folder, output_folder, roi=(50, 200, 50, 250, 50, 200)):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".nrrd"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            img = sitk.ReadImage(input_path)
            arr = sitk.GetArrayFromImage(img)
            cropped_arr = arr[:, roi[0]:roi[1], roi[2]:roi[3]]
            cropped_img = sitk.GetImageFromArray(cropped_arr)
            cropped_img.CopyInformation(img)
            sitk.WriteImage(cropped_img, output_path)
    print("Cropping completed!")

def normalize_image(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".nrrd"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            img = sitk.ReadImage(input_path)
            arr = sitk.GetArrayFromImage(img)
            arr = (arr - np.mean(arr)) / np.std(arr)
            norm_img = sitk.GetImageFromArray(arr)
            norm_img.CopyInformation(img)
            sitk.WriteImage(norm_img, output_path)
    print("Normalization completed!")

def augment_image(image):
    augmentations = Compose([
        RandRotate(range_x=0.1, prob=0.5),
        RandGaussianNoise(prob=0.5, mean=0, std=0.05),
        RandShiftIntensity(offsets=0.1, prob=0.5),
    ])
    return augmentations(image)

def train_nnunet(dataset_id="001"):
    subprocess.run(["nnUNetv2_plan_and_preprocess", "-d", dataset_id, "--verify_dataset_integrity"])
    subprocess.run(["nnUNetv2_train", dataset_id, "3d_fullres", "0"])
    print("Training completed!")

def evaluate_nnunet(dataset_id="001", test_images="path_to_test_images", output_path="path_to_predictions"):
    subprocess.run(["nnUNetv2_predict", "-i", test_images, "-o", output_path, "-d", dataset_id, "-c", "3d_fullres", "-f", "all"])
    print("Evaluation completed!")

if __name__ == "__main__":
    resample_nrrd("raw_nrrd", "resampled_1mm")
    crop_image("resampled_1mm", "cropped")
    normalize_image("cropped", "normalized")
    train_nnunet()
    evaluate_nnunet()
